In [15]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from random import randint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [16]:
camry_path = 'data/training/vehicle_images/camry_binary'
minivan_path = 'data/training/vehicle_images/minivan_binary'
suv_path = 'data/training/vehicle_images/suv_binary'

In [17]:
video = 'vid1.mp4'
vehicle_paths = [camry_path, minivan_path, suv_path]

In [18]:
training_images = []
labels = []

In [19]:
for path in vehicle_paths:
    for image in os.listdir(path):
        image = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE)

        _, binary_image = cv2.threshold(image, 128, 1, cv2.THRESH_BINARY)  
        binary_image = cv2.resize(binary_image, (32,64))
        binary_image = binary_image.reshape(-1,1)

        training_images.append(binary_image)
        labels.append(1)
training_images = np.array(training_images)

In [20]:
blobs = np.array([np.array([randint(0,1) for _ in range(2048)]) for _ in range(len(training_images))])
for _ in blobs:
    labels.append(0)

In [21]:
labels = np.array(labels)
training_images = np.squeeze(training_images, axis=2)

In [22]:
x_train,X_test,y_train, y_test = train_test_split(np.concatenate([training_images, blobs]), labels, test_size=0.3,random_state=102)

In [23]:
model = Sequential()
model.add(Dense(2048, activation='sigmoid'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1))

In [24]:
model.compile(optimizer='rmsprop', loss='mse')

In [25]:
model.fit(x=x_train, y=y_train, epochs=30)

Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - loss: 42733.0469
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - loss: 0.2637
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - loss: 0.2094
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - loss: 0.2125
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - loss: 0.1563
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - loss: 0.0656
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - loss: 0.0336
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 0.2096
Epoch 9/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - loss: 0.0439
Epoch 10/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 0.0216
Epoch 11/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - loss: 0.5549
Epoch 12/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - loss: 0.3085
Epoch 13/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - loss: 0.3804
Epoch 14/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 0.1915
Epoch 15/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - loss: 0.0814
Epoch 16/30
9/9

# VIDEO STUFF

In [130]:
vid = cv2.VideoCapture(f'data/videos/{video}')
back_sub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

In [131]:
vid = cv2.VideoCapture(f'data/videos/{video}')
back_sub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

In [132]:
color_to_exclude = [0, 0, 0]

def get_dominant_color(image, contour):
    mask = np.zeros(image.shape[:2], np.uint8)
    cv2.drawContours(mask, [contour], 0, 255, -1)
    mean_color = cv2.mean(image, mask=mask)

    return mean_color

while (vid.isOpened()):
    # Read a frame from the video
    ret, frame = vid.read()
    if not ret:
        break

    # Apply the background subtractor to the frame
    fg_mask = back_sub.apply(frame)

    # Apply a binary threshold to clean up the mask
    _, binary = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)

    # Morphological operations to remove small noise
    kernel_open = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    kernel_close = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel_open, iterations=2)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel_close, iterations=2)

    # Find contours to identify large moving objects (cars)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a new mask to draw detected cars
    cnum = 0
    mask = np.zeros_like(binary)
    cv2.putText(frame, f"CAR {index+1}: ", (x,y+-0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (r,g,b), 2)
    for index,contour in enumerate(contours):
        # Filter out small contours based on area
        if cv2.contourArea(contour) > 500:  # Adjust this threshold as needed
            x, y, w, h = cv2.boundingRect(contour)
            obj = []
            for i in range(y, (y+h+1)):
                for j in range(x, (x+w+1)):
                    if [j,i] in contour:
                        obj.append(1)
                    else:
                        obj.append(0)
            
            obj = np.array(obj, dtype=np.uint8)
            obj = cv2.resize(obj, (32,64))
            obj = obj.reshape(1,-1)
            
            if model.predict(obj) < 0:
                continue
            color = get_dominant_color(frame,contour)
            print(color)
            r = color[0]
            g = color[1]
            b = color[2]

            
            cnum += 1
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(frame, f"CAR {index+1}: ", (x,y+-0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (r,g,b), 2)
            
    cv2.putText(frame, f"{cnum} cars currently on screen.", (+30,+30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    out.write(frame)
            


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
(127.9740334679746, 122.07616849394114, 122.76341604154645, 0.0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
(156.68843777581642, 150.9082082965578, 151.4819064430715, 0.0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
(115.4751550630706, 110.97560805631055, 107.148721165238, 0.0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
(115.46135619206913, 110.9674541779915, 107.11582688689107, 0.0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
(116.17165301175244, 111.22166988861274, 108.940347970174, 0.0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(116.4543865030675, 111.53242331288344, 109.22322085889571, 0.0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
(116.64409443964985, 111.72004068548883, 109.400967821477, 0.0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
(117.0950408332347, 112.50775239673335, 110.02165936797255, 0.0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
(107.90016960894388, 102.84558419531307, 101.73476505577985, 0.0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
(

In [133]:
out.release()